In [1]:
# import lib
import mediapipe as mp
import cv2 as cv
import json
import os
import time
import gc
import numpy as np

In [2]:
# construct mp solutions

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
sign_data = {}

In [7]:
# setup loop

# Specify the directory
directory = '../sibi/'

# Get a list of all files in the directory
signs = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]

# print(len(signs))

# signs = ['Ipar [Ipar].webm', 'A [A].webm', 'B [B].webm']
# landmarker = mp_holistic.Holistic(static_image_mode=True,
#                                   model_complexity=1,
#                                   min_detection_confidence=0.5)

pose_landmarker = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.7, min_tracking_confidence=0.7,
                               smooth_landmarks=True, model_complexity=0)

start_time = time.time()
sign_meta = {}

In [8]:
directory = '../data/raw/pose'
extracted = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]
print(len(signs), len(extracted))
signs = signs[1553:]

3394 3394


In [5]:
# loop through every file

print("- Extract Raw Data -")
for i in range(len(signs)):
    start_per_sign_time = time.time()
    sign = signs[i] # .split('[')[1][0:-6]
    sign_name = sign.split('[')[1][0:-6]

    print(f"File: {sign}")

    vid = '../sibi/' + sign
    cap = cv.VideoCapture(vid)

    current_frame = 0
    frames = []
    while cap.isOpened():
        ret, image = cap.read()

        if not ret:
            break

        image = cv.flip(image, 1)

        upscaled_img = cv.resize(image, None, fx=2, fy=2, interpolation=cv.INTER_CUBIC)

        frame = cv.cvtColor(upscaled_img, cv.COLOR_BGR2RGB)
        frame.flags.writeable = False
        pose_results = pose_landmarker.process(frame)
        frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)

        # pose
        pose_world = pose_results.pose_world_landmarks
        temp_pose = {}
        if pose_world:
            for idx, landmark in enumerate(pose_world.landmark):
                if idx in [11, 12, 13, 14, 15, 16]:
                    xyz = {"x": landmark.x,
                           "y": -landmark.y,
                           "z": -landmark.z,
                           "visibility": landmark.visibility}
                    temp_pose[idx] = xyz

        if not temp_pose:
            temp_pose = None

        # into one
        frames.append({
            "frame_idx": current_frame,
            "pose": temp_pose})

        current_frame += 1

        del temp_pose

    end_per_sign_time = time.time()
    delta_per_sign_time = end_per_sign_time - start_per_sign_time
    sign_meta[sign] = {"time": delta_per_sign_time,
                       "frame_count": current_frame}

    print(f"Time: {delta_per_sign_time} seconds\n")

    sign_frame = {
        'metadata': {
            'sign': sign_name,
            'time': delta_per_sign_time,
            'frame_count': current_frame
        },
        'frames': frames
    }

    with open(f"../data/raw/pose/{sign_name}.json", "w") as json_file:
        json_file.write(json.dumps(sign_frame, indent=4))
    json_file.close()

    del frames

    if i % 500 == 0:
        gc.collect()

end_time = time.time()
delta_time = end_time - start_time
print(f"Finished in {delta_time} seconds.")

- Extract Raw Data -
File: Kopi [Kopi].webm
Time: 10.20694375038147 seconds

File: Koran [Koran].webm
Time: 8.467329978942871 seconds

File: Korek [Korek].webm
Time: 5.9164745807647705 seconds

File: Kosong [Kosong].webm
Time: 8.419810771942139 seconds

File: Kota [Kota].webm
Time: 10.531467199325562 seconds

File: Kotak [Kotak].webm
Time: 10.312429189682007 seconds

File: Kotor [Kotor].webm
Time: 8.180089712142944 seconds

File: Koyak [Koyak].webm
Time: 7.801802635192871 seconds

File: Kredit [Kredit].webm
Time: 8.843825101852417 seconds

File: Kristen [Kristen].webm
Time: 7.048089265823364 seconds

File: Kuah [Kuah].webm
Time: 8.981092929840088 seconds

File: Kuasa [Kuasa].webm
Time: 7.665347576141357 seconds

File: Kuat [Kuat].webm
Time: 9.31778597831726 seconds

File: Kubik [Kubik].webm
Time: 10.581551313400269 seconds

File: Kubur [Kubur].webm
Time: 6.8820109367370605 seconds

File: Kucing [Kucing].webm
Time: 8.17912745475769 seconds

File: Kuda [Kuda].webm
Time: 6.745166540145874

In [6]:
# print(sign_meta)
# print(delta_time)
meta_data = {
    "total_time": delta_time,
    "meta": sign_meta,
}

with open(f"../data/meta/meta_data_raw_pose.json", "w") as meta_data_file:
    meta_data_file.write(json.dumps(meta_data, indent=4))
meta_data_file.close()